In [42]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    c=2*rand(Float64,length(v)).-1
    #c=rand(Float64,length(v))
    return c'*v
end
# random quadratic objective function f
v=reverse(monomials(x,0:5))
f=generate_random_poly(v)
#f=f^2


# unit sphere constraint
m=1
q=floor(Int64,n/m)
R=ones(Float64,m)./m
T=[(j-1)*q+1:j*q for j in 1:m-1]
append!(T,[(m-1)*q+1:n])

g=[R[j]-sum(x[T[j]].^2) for j in 1:m]

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=[2*rand(length(T[j])).-1 for j in 1:m]# create a feasible solution
randx=[sqrt(R[j])*rand(1)[1]*randx[j]/norm(randx[j]) for j in 1:m]
randx=vcat(randx...)

for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)

***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=2


In [49]:
k=4
println("Relaxed order: k=",k)

Relaxed order: k=4


In [50]:
using ctpPOP

opt_val0=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3)

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1287            
  Cones                  : 0               
  Scalar variables       : 43              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

-0.8570078346220894

In [51]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

opt_val1=ctpPOP.POP_dense_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
                         maxit=Int64(1e6),tol=1e-3,
                         use_eqcons_to_get_constant_trace=false,
                         check_tol_each_iter=true)
println("gap = ",abs(opt_val1-opt_val0)/abs(opt_val0)*100)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 5.0
  Number of blocks: omega=2
  Size of the largest block: s^max=126
  Number of equality trace constraints: zeta=8353
Modeling time:
  0.092508 seconds (364.08 k allocations: 50.382 MiB, 31.33% gc time)
iter=1   val=-6.735585557055925   gap=0.0001047791854641555   feas=6.132736333786388
iter=2   val=-5.702717383546906   gap=0.07821563722243842   feas=4.55831530860276
iter=4   val=-4.83381958613961   gap=0.20738328453676277   feas=3.1742427726810347
iter=8   val=-3.093058133646316   gap=0.18789723337115258   feas=1.699857469433972
iter=16   val=-2.199764129404866   gap=0.21516494718911625   feas=0.8855809305815392
iter=32   val=-1.667439020846604   gap=0.10932728698788492   feas=0.3817455493686524
iter=64   val=-1.4926534840968229   gap=0.09181026610696559   feas=0.2407850670130008
iter=128   val=-1.3714805939989083   gap=0.1146333416421032   feas=0.17191817418250321
iter=256   val=-1.230440528045774   gap=0.1236576529

In [8]:
using ctpPOP

opt_val2=ctpPOP.POP_dense_SOS(x,f,g,h,k,tol=1e-3,solver="COSMO")

println("gap = ",abs(opt_val2-opt_val0)/abs(opt_val0)*100)

------------------------------------------------------------------
          COSMO v0.7.8 - A Quadratic Objective Conic Solver
                         Michael Garstka
                University of Oxford, 2017 - 2020
------------------------------------------------------------------

Problem:  x ∈ R^{255},
          constraints: A ∈ R^{378x255} (722 nnz),
          matrix size to factor: 633x633,
          Floating-point precision: Float64
Sets:     PsdConeTriangle of dim: 231
          ZeroSet of dim: 126
          PsdConeTriangle of dim: 21
Settings: ϵ_abs = 1.0e-04, ϵ_rel = 1.0e-04,
          ϵ_prim_inf = 1.0e-06, ϵ_dual_inf = 1.0e-04,
          ρ = 0.1, σ = 1e-06, α = 1.6,
          max_iter = 2500,
          scaling iter = 10 (on),
          check termination every 40 iter,
          check infeasibility every 40 iter,
          KKT system solver: COSMO.QdldlKKTSolver
Setup Time: 837.51ms

Iter:	Objective:	Primal Res:	Dual Res:	Rho:
1	-2.6789e+01	1.6012e+01	5.9998e-01	1.0000e-01
4

In [16]:
# using ctpPOP

# opt_val=ctpPOP.POP_dense_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
#                          tol=1e-3,use_eqcons_to_get_constant_trace=false)

In [24]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)